In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)
from scipy import stats
from scipy.stats import norm, skew #for some statistics
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x))

In [33]:
train = pd.read_csv('train-new.csv')
train.head()

,dvcat,weight,dead,airbag,seatbelt,frontal,sex,ageOFocc,yearacc,yearVeh,abcat,occRole,deploy,caseid,injSeverity
0,55+,53.342,dead,airbag,belted,1,f,48,2002,1997,deploy,driver,1,45:150:1,3
1,25-39,154.960,alive,none,none,1,m,26,2001,1968,unavail,driver,0,76:40:1,3
2,55+,38.994,alive,none,none,1,f,51,2002,1994,unavail,driver,0,11:184:1,3
3,25-39,168.568,alive,airbag,belted,1,m,27,1998,1996,deploy,pass,1,9:17:1,3
4,10-24,27.751,alive,airbag,belted,0,m,26,2002,1997,nodeploy,pass,0,2:2:2,0


In [34]:
train.isnull().sum()
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17565 entries, 0 to 17564
Data columns (total 15 columns):
dvcat          17565 non-null object
weight         17565 non-null float64
dead           17565 non-null object
airbag         17565 non-null object
seatbelt       17565 non-null object
frontal        17565 non-null int64
sex            17565 non-null object
ageOFocc       17565 non-null int64
yearacc        17565 non-null int64
yearVeh        17565 non-null int64
abcat          17565 non-null object
occRole        17565 non-null object
deploy         17565 non-null int64
caseid         17565 non-null object
injSeverity    17565 non-null int64
dtypes: float64(1), int64(6), object(8)
memory usage: 2.0+ MB


In [35]:
train.injSeverity.value_counts()

3    5718
0    4375
1    3740
2    2912
4     729
5      90
6       1
Name: injSeverity, dtype: int64

In [36]:
test = pd.read_csv('test-new.csv')
test.isnull().sum()

dvcat       0
weight      0
dead        0
airbag      0
seatbelt    0
frontal     0
sex         0
ageOFocc    0
yearacc     0
yearVeh     1
abcat       0
occRole     0
deploy      0
caseid      0
dtype: int64

In [38]:
ntrain = train.shape[0]
ntest = test.shape[0]
y_train = train['injSeverity']
all_data = pd.concat((train,test)).reset_index(drop = True)
all_data = all_data.drop('injSeverity',axis = 1)
print("all_data size is : {}".format(all_data.shape))

all_data size is : (26217, 14)


In [41]:
all_data.dvcat.value_counts()

10-24    12848
25-39     8214
40-54     2977
55        1492
1-9        686
Name: dvcat, dtype: int64

In [40]:
all_data['dvcat'] = np.where(all_data['dvcat'].str[2] == '+','55',all_data['dvcat'])
all_data['dvcat'] = all_data['dvcat'].str.replace('km/h','')

In [42]:
k=all_data['dvcat'].str.split("-",expand=True).astype(float)

all_data['dv']=0.5*(k[0]+k[1])

In [44]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26217 entries, 0 to 26216
Data columns (total 15 columns):
abcat       26217 non-null object
ageOFocc    26217 non-null int64
airbag      26217 non-null object
caseid      26217 non-null object
dead        26217 non-null object
deploy      26217 non-null int64
dvcat       26217 non-null object
frontal     26217 non-null int64
occRole     26217 non-null object
seatbelt    26217 non-null object
sex         26217 non-null object
weight      26217 non-null float64
yearVeh     26216 non-null float64
yearacc     26217 non-null int64
dv          24725 non-null float64
dtypes: float64(3), int64(4), object(8)
memory usage: 3.0+ MB


In [45]:
all_data['dv'] = all_data['dv'].fillna(55)

In [46]:
all_data_na = (all_data.isnull().sum()/len(all_data))*100
all_data_na = all_data_na.drop(all_data_na[all_data_na ==0].index).sort_values(ascending = False)
missing_data = pd.DataFrame({'missing rate':all_data_na})
missing_data

,missing rate
yearVeh,0.004


In [47]:
all_data[all_data.yearVeh.isnull()]

,abcat,ageOFocc,airbag,caseid,dead,deploy,dvcat,frontal,occRole,seatbelt,sex,weight,yearVeh,yearacc,dv
19236,deploy,27,airbag,11:138:1,alive,1,10-24,1,driver,belted,m,8521.810,nan,1997,17.000


In [52]:
all_data['yearVeh'] = all_data['yearVeh'].fillna(1980)

In [53]:
all_data.isnull().sum()

abcat       0
ageOFocc    0
airbag      0
caseid      0
dead        0
deploy      0
dvcat       0
frontal     0
occRole     0
seatbelt    0
sex         0
weight      0
yearVeh     0
yearacc     0
dv          0
dtype: int64

In [54]:
all_data = all_data.drop(['caseid','dvcat'],axis = 1)

In [55]:
all_data.head()

,abcat,ageOFocc,airbag,dead,deploy,frontal,occRole,seatbelt,sex,weight,yearVeh,yearacc,dv
0,deploy,48,airbag,dead,1,1,driver,belted,f,53.342,1997.000,2002,55.000
1,unavail,26,none,alive,0,1,driver,none,m,154.960,1968.000,2001,32.000
2,unavail,51,none,alive,0,1,driver,none,f,38.994,1994.000,2002,55.000
3,deploy,27,airbag,alive,1,1,pass,belted,m,168.568,1996.000,1998,32.000
4,nodeploy,26,airbag,alive,0,0,pass,belted,m,27.751,1997.000,2002,17.000


In [56]:
numerical_feat = all_data.dtypes[all_data.dtypes != 'object'].index
skewed_feat = all_data[numerical_feat].apply(lambda x: skew(x.dropna())).sort_values(ascending = False)
skewness = pd.DataFrame(skewed_feat,columns = ['skew'])
skewness

,skew
weight,14.310
ageOFocc,0.904
dv,0.719
deploy,0.690
yearacc,-0.011
frontal,-0.598
yearVeh,-0.896


In [57]:
from scipy.special import boxcox1p
lam = 0.15
skewness = skewness[skewness['skew'] > 0.75]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))
skewness_feat = skewness.index
for i in skewness_feat:
    all_data[i] = boxcox1p(all_data[i],lam)

There are 2 skewed numerical features to Box Cox transform


In [58]:
all_data.head()

,abcat,ageOFocc,airbag,dead,deploy,frontal,occRole,seatbelt,sex,weight,yearVeh,yearacc,dv
0,deploy,5.285,airbag,dead,1,1,driver,belted,f,5.472,1997.000,2002,55.000
1,unavail,4.263,none,alive,0,1,driver,none,m,7.552,1968.000,2001,32.000
2,unavail,5.392,none,alive,0,1,driver,none,f,4.927,1994.000,2002,55.000
3,deploy,4.323,airbag,alive,1,1,pass,belted,m,7.732,1996.000,1998,32.000
4,nodeploy,4.263,airbag,alive,0,0,pass,belted,m,4.367,1997.000,2002,17.000


In [59]:
all_data = pd.get_dummies(all_data)
print(all_data.shape)

(26217, 20)


In [60]:
train = all_data[:ntrain]
test = all_data[ntrain:]

In [61]:
print(train.shape)
print(test.shape)

(17565, 20)
(8652, 20)


In [62]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold,cross_val_score
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,roc_auc_score
import xgboost as xgb
import lightgbm as lgb
from sklearn.naive_bayes import GaussianNB

In [63]:
lr = LogisticRegression()
dt = DecisionTreeClassifier(criterion='entropy',random_state=0)
rf = RandomForestClassifier(n_estimators=100,random_state=0)
nb = GaussianNB()
nb_bag = BaggingClassifier(base_estimator=nb,n_estimators=10,random_state=0)
lr_bag = BaggingClassifier(base_estimator=lr,n_estimators=10,random_state=0)
dt_bag = BaggingClassifier(base_estimator=dt,n_estimators=10,random_state=0)
nb_boost = AdaBoostClassifier(base_estimator=nb,n_estimators=50,random_state=0)
lr_boost = AdaBoostClassifier(base_estimator=lr,n_estimators=50,random_state=0)
dt_boost = AdaBoostClassifier(base_estimator=dt,n_estimators=50,random_state=0)
rf_boost = AdaBoostClassifier(base_estimator=rf,n_estimators=50,random_state=0)
gboost = GradientBoostingClassifier(n_estimators=49,random_state = 0)
LR_bag_best = BaggingClassifier(base_estimator=lr,n_estimators=81)

In [64]:
models = []
models.append(('logit',lr))
models.append(('dt',dt))
models.append(('RandomForest',rf))
models.append(('NB',nb))
models.append(('nb_bag',nb_bag))
models.append(('lr_bag',lr_bag))
models.append(('dt_bag',dt_bag))
models.append(('nb_boost',nb_boost))
models.append(('lr_boost',lr_boost))
models.append(('dt_boost',dt_boost))
models.append(('rf_boost',rf_boost))
models.append(('gboost',gboost))
models.append(('LR_bag_best',LR_bag_best))

In [66]:
results = []
names = []
for name,model in models:
    kfold = KFold(n_splits=15,shuffle=True,random_state=42).get_n_splits(train.values)
    cv_results = cross_val_score(model,train,y_train,cv = kfold,scoring='accuracy')
    results.append(cv_results)
    names.append(name)
    print("%s: %f (%f)" %(name, np.mean(cv_results),np.std(cv_results,ddof = 1)))

logit: 0.426644 (0.013533)
dt: 0.405010 (0.013832)
RandomForest: 0.469798 (0.013319)
NB: 0.283746 (0.011720)
nb_bag: nan (nan)
lr_bag: 0.431369 (0.010772)
dt_bag: 0.456988 (0.013463)
nb_boost: 0.265983 (0.023378)
lr_boost: 0.377114 (0.050898)
dt_boost: 0.473783 (0.011878)
rf_boost: 0.417364 (0.069117)
gboost: 0.512440 (0.009424)
LR_bag_best: 0.433931 (0.009607)


In [67]:
from sklearn.metrics import accuracy_score 
gboost.fit(train,y_train)
pred_train = gboost.predict(train)
pred_train_prob = gboost.predict_proba(train)
pred_test = gboost.predict(test)
accuracy_score(y_train,pred_train)

0.5263307714204384

In [68]:
pred1 = pd.DataFrame(pred_test)
pred1.to_csv('hacka_submission1.csv')

In [69]:
import keras

Using TensorFlow backend.


In [77]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
train1 = sc.fit_transform(train)
test1 = sc.transform(test)

In [95]:
import keras
from keras.models import Sequential
from keras.layers import Dense

# Initialising the ANN
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 14, kernel_initializer = 'uniform', activation = 'relu', input_dim = 20))

# Adding the second hidden layer
classifier.add(Dense(units = 14, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 7, kernel_initializer = 'uniform', activation = 'softmax'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(train, y_train, batch_size =200 , epochs = 500)

Epoch 1/500
17565/17565 [==============================] - 0s 15us/step - loss: 1.5531 - accuracy: 0.3106
Epoch 2/500
17565/17565 [==============================] - 0s 7us/step - loss: 1.5011 - accuracy: 0.3255
Epoch 3/500
17565/17565 [==============================] - 0s 8us/step - loss: 1.5002 - accuracy: 0.3255
Epoch 4/500
17565/17565 [==============================] - 0s 7us/step - loss: 1.4972 - accuracy: 0.3256
Epoch 5/500
17565/17565 [==============================] - 0s 7us/step - loss: 1.4862 - accuracy: 0.3318
Epoch 6/500
17565/17565 [==============================] - 0s 7us/step - loss: 1.4494 - accuracy: 0.3847
Epoch 7/500
17565/17565 [==============================] - 0s 7us/step - loss: 1.3770 - accuracy: 0.4204
Epoch 8/500
17565/17565 [==============================] - 0s 7us/step - loss: 1.3479 - accuracy: 0.4230
Epoch 9/500
17565/17565 [==============================] - 0s 7us/step - loss: 1.3302 - accuracy: 0.4262
Epoch 10/500
17565/17565 [============================

17565/17565 [==============================] - 0s 9us/step - loss: 1.1888 - accuracy: 0.4958
Epoch 155/500
17565/17565 [==============================] - 0s 8us/step - loss: 1.1903 - accuracy: 0.4934
Epoch 156/500
17565/17565 [==============================] - 0s 8us/step - loss: 1.1900 - accuracy: 0.4935
Epoch 157/500
17565/17565 [==============================] - 0s 8us/step - loss: 1.1921 - accuracy: 0.4926
Epoch 158/500
17565/17565 [==============================] - 0s 6us/step - loss: 1.1949 - accuracy: 0.4899
Epoch 159/500
17565/17565 [==============================] - 0s 7us/step - loss: 1.1970 - accuracy: 0.4898
Epoch 160/500
17565/17565 [==============================] - 0s 7us/step - loss: 1.1882 - accuracy: 0.4944
Epoch 161/500
17565/17565 [==============================] - 0s 7us/step - loss: 1.1875 - accuracy: 0.4943
Epoch 162/500
17565/17565 [==============================] - 0s 6us/step - loss: 1.1870 - accuracy: 0.4980
Epoch 163/500
17565/17565 [========================

17565/17565 [==============================] - 0s 12us/step - loss: 1.1793 - accuracy: 0.5005
Epoch 306/500
17565/17565 [==============================] - 0s 10us/step - loss: 1.1778 - accuracy: 0.5027
Epoch 307/500
17565/17565 [==============================] - 0s 7us/step - loss: 1.1772 - accuracy: 0.5037
Epoch 308/500
17565/17565 [==============================] - 0s 6us/step - loss: 1.1821 - accuracy: 0.4992
Epoch 309/500
17565/17565 [==============================] - 0s 7us/step - loss: 1.1792 - accuracy: 0.5023
Epoch 310/500
17565/17565 [==============================] - 0s 6us/step - loss: 1.1843 - accuracy: 0.5004
Epoch 311/500
17565/17565 [==============================] - 0s 7us/step - loss: 1.1822 - accuracy: 0.5002
Epoch 312/500
17565/17565 [==============================] - 0s 6us/step - loss: 1.1802 - accuracy: 0.5009
Epoch 313/500
17565/17565 [==============================] - 0s 7us/step - loss: 1.1816 - accuracy: 0.5013
Epoch 314/500
17565/17565 [======================

17565/17565 [==============================] - 0s 7us/step - loss: 1.1805 - accuracy: 0.5008
Epoch 458/500
17565/17565 [==============================] - 0s 6us/step - loss: 1.1817 - accuracy: 0.5025
Epoch 459/500
17565/17565 [==============================] - 0s 6us/step - loss: 1.1760 - accuracy: 0.5053
Epoch 460/500
17565/17565 [==============================] - 0s 7us/step - loss: 1.1779 - accuracy: 0.5018
Epoch 461/500
17565/17565 [==============================] - 0s 7us/step - loss: 1.1765 - accuracy: 0.5038
Epoch 462/500
17565/17565 [==============================] - 0s 7us/step - loss: 1.1772 - accuracy: 0.5066
Epoch 463/500
17565/17565 [==============================] - 0s 7us/step - loss: 1.1765 - accuracy: 0.5034
Epoch 464/500
17565/17565 [==============================] - 0s 6us/step - loss: 1.1754 - accuracy: 0.5064
Epoch 465/500
17565/17565 [==============================] - 0s 6us/step - loss: 1.1769 - accuracy: 0.5045
Epoch 466/500
17565/17565 [========================

In [96]:
model_xgb = xgb.XGBClassifier(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=10, 
                             min_child_weight=1.7817, n_estimators=1500,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

In [97]:
model_xgb.fit(train,y_train)
pred_train = model_xgb.predict(train)
pred_test = model_xgb.predict(test)
accuracy_score(y_train,pred_train)

[15:10:54] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




0.565670367207515

In [98]:
pred2 = pd.DataFrame(pred_test)
pred2.to_csv('hacka_submission2.csv')

In [100]:
auc_avg = []
for i in range(1,100):
    model_xgb_def = xgb.XGBClassifier(n_estimators=i,random_state=0)
    kfold = KFold(shuffle = True, n_splits = 15, random_state = 0)
    results = cross_val_score(model_xgb_def,train,y_train,cv = kfold, scoring = 'accuracy')
    auc_avg.append(np.mean(results))
print(auc_avg)

[0.5054369484770852, 0.5088528323370338, 0.511585539424993, 0.509706803302021, 0.5111300882436663, 0.5121548534016509, 0.5105038428693425, 0.5113578138343297, 0.5118132650156562, 0.5110162254483347, 0.5111300882436665, 0.5110162254483347, 0.5109023626530032, 0.511528608027327, 0.5123825789923143, 0.512496441787646, 0.5113008824366638, 0.5119840592086535, 0.5122687161969826, 0.5124964417876459, 0.5126672359806432, 0.5130088243666381, 0.5125533731853117, 0.5126672359806433, 0.5128949615713067, 0.5132934813549672, 0.512781098775975, 0.5124964417876459, 0.5130088243666382, 0.5123825789923143, 0.5125533731853117, 0.5125533731853117, 0.5128380301736407, 0.5120979220039852, 0.5119271278109878, 0.5110162254483348, 0.5107884998576715, 0.5113008824366639, 0.5102761172786793, 0.510617705664674, 0.50999146029035, 0.5103330486763451, 0.5110731568460005, 0.5107884998576715, 0.5101622544833476, 0.5099914602903503, 0.5103330486763451, 0.5097637346996868, 0.5093082835183603, 0.5090805579276972, 0.50890